# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
!mongoimport --type json -d uk_food -c establishments --drop --jsonArray ./Resources/establishments.json

2024-01-05T12:59:51.180-0600	connected to: mongodb://localhost/
2024-01-05T12:59:51.184-0600	dropping: uk_food.establishments
2024-01-05T12:59:52.474-0600	39779 document(s) imported successfully. 0 document(s) failed to import.


In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
mongo.list_database_names()

['admin',
 'autosarus',
 'classDB',
 'config',
 'epa',
 'fruits_db',
 'local',
 'met',
 'uk_food']

In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
db.collection_names()

C:\Users\skvar\AppData\Local\Temp\ipykernel_16924\1966569860.py:2: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  db.collection_names()


['establishments']

In [7]:
# review a document in the establishments collection
db.establishments.find_one()

{'_id': ObjectId('659851a70627de53812c1b5e'),
 'FHRSID': 551803,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000070948',
 'BusinessName': 'The Tea Room',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'PostCode': 'CT15 6DZ',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2021-08-17T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.383298', 'latitude': '51.148133'},
 'RightToReply': '',
 'Distance': 4587.362402580997,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
restaurant_details = {
    'BusinessName': 'Penang Flavours',
    'BusinessType': 'Restaurant/Cafe/Canteen',
    'RatingValue': 'None'
    
}

In [10]:
# Insert the new restaurant into the collection
establishments.insert_one(restaurant_details)

In [11]:
# Check that the new restaurant was inserted
results = establishments.find(restaurant_details)
for result in results:
    print(result)

{'_id': ObjectId('659851baec8bf4b7dac5e57d'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'RatingValue': 'None'}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
from pprint import pprint

In [13]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': {'$in': ['Restaurant/Cafe/Canteen']}}
field = {'BusinessTypeID': 1}
results = list(establishments.find(query, field))
pprint(results)

[{'BusinessTypeID': 1, '_id': ObjectId('659851a70627de53812c1b5e')},
 {'BusinessTypeID': 1, '_id': ObjectId('659851a70627de53812c1b63')},
 {'BusinessTypeID': 1, '_id': ObjectId('659851a70627de53812c1b64')},
 {'BusinessTypeID': 1, '_id': ObjectId('659851a70627de53812c1b65')},
 {'BusinessTypeID': 1, '_id': ObjectId('659851a70627de53812c1b6f')},
 {'BusinessTypeID': 1, '_id': ObjectId('659851a70627de53812c1b71')},
 {'BusinessTypeID': 1, '_id': ObjectId('659851a70627de53812c1b75')},
 {'BusinessTypeID': 1, '_id': ObjectId('659851a70627de53812c1b77')},
 {'BusinessTypeID': 1, '_id': ObjectId('659851a70627de53812c1b78')},
 {'BusinessTypeID': 1, '_id': ObjectId('659851a70627de53812c1b7a')},
 {'BusinessTypeID': 1, '_id': ObjectId('659851a70627de53812c1b7b')},
 {'BusinessTypeID': 1, '_id': ObjectId('659851a70627de53812c1b7c')},
 {'BusinessTypeID': 1, '_id': ObjectId('659851a70627de53812c1b80')},
 {'BusinessTypeID': 1, '_id': ObjectId('659851a70627de53812c1b8c')},
 {'BusinessTypeID': 1, '_id': Obje

3. Update the new restaurant with the `BusinessTypeID` you found.

In [14]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": 1}}
)

In [15]:
# Confirm that the new restaurant was updated
query =({"BusinessName": "Penang Flavours"})
establishments.find_one(query)

{'_id': ObjectId('659851baec8bf4b7dac5e57d'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'RatingValue': 'None',
 'BusinessTypeID': 1}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [16]:
# Find how many documents have LocalAuthorityName as "Dover"
query ={'LocalAuthorityName': 'Dover'}
count = establishments.count_documents(query)
print(count)

994


In [17]:
# Delete all documents where LocalAuthorityName is "Dover"
query ={'LocalAuthorityName': 'Dover'}
establishments.delete_many(query)

In [18]:
# Check if any remaining documents include Dover
results = establishments.find(query)
for result in results:
    print(result)

In [19]:
# Check that other documents remain with 'find_one'
results = establishments.find_one(query)
print(results)

None


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [20]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [{'$set':{'geocode.longitude':{'$toDouble': '$geocode.longitude'}}}, {'$set':{'geocode.latitude':{'$toDouble': '$geocode.latitude'}}}]
)

Use `update_many` to convert `RatingValue` to integer numbers.

In [21]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [28]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({},
                           [{'$set': 
                             {'RatingValue':
                              {'$convert': {'input': '$RatingValue', 
                                            'to': 'int', 
                                            'onError': '', 
                                            'onNull': ''}}}}])


In [29]:
# Check that the coordinates and rating value are now numbers
df_result = pd.DataFrame(results)
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame
